In [6]:
!pip install wget

  Using cached wget-3.2-py3-none-any.whl


In [53]:
import torch
import torch.nn as nn
import wget

In [3]:
batch_size = 4
block_size = 8

# torch.randint(1000000- block_size, (batch_size,))  #randint(low, (size=4,1))

tensor([617986,  27814, 356481, 870909])

In [9]:
file = wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

100% [..........................................................................] 1115394 / 1115394

In [10]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [25]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda s: ''.join(itos[c] for c in s)

In [27]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)*0.8)
train_data = data[:n]
test_data = data[n:]

In [28]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xbatch, ybatch = get_batch('train')

# Attention

In [46]:
t = torch.softmax(torch.tril(torch.ones(4,4)), 1)
# t.sum()
t

tensor([[0.4754, 0.1749, 0.1749, 0.1749],
        [0.3655, 0.3655, 0.1345, 0.1345],
        [0.2969, 0.2969, 0.2969, 0.1092],
        [0.2500, 0.2500, 0.2500, 0.2500]])

In [51]:
torch.softmax(torch.tril(torch.ones(4,4)).masked_fill(torch.tril(torch.ones(4,4)) == 0, float('-inf')), 1)

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])

NameError: name 'x' is not defined

In [49]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))

In [6]:
stringToInt = {s:i for i,s in enumerate(chars)}
intToString = {i:s for i,s in enumerate(chars)}

In [16]:
encode = lambda s: [stringToInt[c] for c in s]
decode = lambda i: ''.join([intToString[c] for c in i])

In [14]:
encode("hi")

[46, 47]

In [17]:
decode([46,47])

'hi'

In [31]:
data = torch.tensor(encode(text), dtype=torch.long)

In [36]:
n = int(0.9*len(text))
train = data[:n]
test = data[n:]

In [33]:
batch_size = 4
block_size = 8

In [37]:
def get_batch():
    data = train
    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x,y

In [38]:
xb, yb = get_batch()

In [39]:
xb

tensor([[ 7, 50, 47, 49, 43,  1, 43, 62],
        [53, 56,  1, 50, 47, 44, 43,  1],
        [15, 20, 13, 30, 16,  1, 21, 21],
        [47, 52, 53, 59, 57,  1, 39, 52]])

In [40]:
vocab_size = len(chars)

In [50]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, x, y=None):
        logits = self.token_embedding_table(x) # (B,T,C)

        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [51]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.9112, grad_fn=<NllLossBackward0>)

uAl$?'K&BlS&-VX!qNE-ggC
D!yubCzKbqUDLJes:-a$enje?!FS,AYgJ;WKWKr?yqUDH' TguBZ?J!ThMReGBFbgXP;Ou: wK;&


In [52]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [53]:
for steps in range(100):
    xb, yb = get_batch()
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

4.552437782287598
